## Implementation of Camera Calibration through OpenCV

In [41]:
# Imports
import cv2
import numpy as np
# import os 
import glob

## Define dimensions of checkerboard

In [42]:
# Defining the dimensions of checkerboard
 # 9,6 to begin with

# Each tile is 7x7 mm
chekcerboard_tile_size = 7 #mm

CHECKERBOARD = (4,4) # Vertical, horizontal
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

## Define world coordinates for 3D points

In [43]:
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

## Extract path of individual image stored in ChessBoards folder

In [46]:
# Need to calibrate with more than one image ideally. 
images = glob.glob('./Images/ChessBoards/LimitedBoards2/*.bmp') 
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected, we refine the pixel coordinates and display them on the images of checker board
    """
    
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()

h,w = img.shape[:2]
 

## Camera calibration using OpenCV
* Zhang based calibration


In [45]:
"""
Performing camera calibration by passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the detected corners (imgpoints)
"""

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


print("Camera matrix : \n")
print(mtx)
print("\n dist : \n")
print(dist)
print("\n rvecs : \n")
print(rvecs)
print("\n tvecs : \n")
print(tvecs)

Camera matrix : 

[[4.13922090e+03 0.00000000e+00 3.22320257e+02]
 [0.00000000e+00 4.18974476e+03 2.45865771e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 dist : 

[[-4.79969522e+00  3.10243894e+03 -2.01999506e-02 -2.33813766e-02
   1.27655739e+01]]

 rvecs : 

(array([[-0.15064384],
       [ 0.01118792],
       [-0.00286632]]), array([[-0.15712447],
       [ 0.03586678],
       [ 0.38804825]]), array([[ 0.13030881],
       [ 0.01566151],
       [-0.03903251]]), array([[ 0.14234059],
       [ 0.03137642],
       [-0.14062137]]), array([[ 0.12668345],
       [-0.00510366],
       [ 0.2021847 ]]))

 tvecs : 

(array([[-1.22658955],
       [-0.91221106],
       [69.49746313]]), array([[-0.5243562 ],
       [-1.97960022],
       [69.59578979]]), array([[-1.2827288 ],
       [-0.80550459],
       [70.51531702]]), array([[-1.46670374],
       [-0.69734627],
       [70.44735871]]), array([[-1.03732726],
       [-1.28759936],
       [70.47840352]]))
